<b> Chapter 4 is a technical chapter which will go over the mathematics behind some of the machine learning models that have been used so far.  I will be focusing on the linear algebra definitions and applications as they more succinctly embody the theories and calculations </b>

# Linear Regression

<b> Simply put linear regression is a weighted average of features and a scalar bias.  The weights are bias are chosen based off the minimization of a cost function.  The most common cost function for linear regressions is Mean Squared Error (MSE) which is rooted (RMSE) when comparing across multiple models.  The reason MSE is used is because the derivative is much simpler to compute and requires significantly less computational power than the rooted version.  Since rooting the function simply scales it, selecting MSE to minimize has the same end result as selected RMSE to minimize.  This is true in general for any cost or reward function and it is encouraged to use the form of the cost function which minimizes computational effort. </b>

## The Normal Equation

#### ${\hat{\theta}}$ = $(\mathbf{X^T}\mathbf{X})^{-1} \mathbf{X^T} \mathbf{y}$

In this equation:

${\hat{\theta}}$ is the value of $\theta$ that minimizes the cost function
<br>
$\mathbf{y}$ is the vector of target values
<br>
$\mathbf{X}$ is a matrix of features with a bias column

In [1]:
' Generate some data to validate the above expression'
import numpy as np

X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1) # mx + b : 3x + 4 + noise

' Introduce the constant bias'
X_b = np.c_[np.ones((100, 1)), X] # m x n matrix with column of 1's and column of X values

' Apply the normal equation'
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
theta_best

array([[3.91900882],
       [3.27450507]])

<b> The bias chosen was 4 and the weight was 3.  The gaussian noise introduced makes it impossible to get these exact values but the derived values are almost perfect.</b>

In [2]:
' Make some predictions'
X_new = np.array([[0], [2]])
X_new_b = np.c_[np.ones((2, 1)), X_new]

y_predict = X_new_b.dot(theta_best)
y_predict

array([[ 3.91900882],
       [10.46801896]])

In [3]:
' Plot the raw data and the best fit predictions'
import matplotlib.pyplot as plt

plt.scatter(X, y)
plt.plot(X, X_b.dot(theta_best), color='orange')

In [4]:
' Performing linear regression with sklearn is very simple'
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X, y)

print('Bias: ', lin_reg.intercept_, 'Coefficient: ', lin_reg.coef_)

lin_reg.predict(X_new)

Bias:  [3.91900882] Coefficient:  [[3.27450507]]


array([[ 3.91900882],
       [10.46801896]])

<b> A caveat for inverted matrices is that they require a non-zero determinant.  This isn't always the case.  A work-around is the Moore-Penrose Inverse aka the pseudoinverse.  This utilizes Singular Value Decomposition to generate an equivalent matrix which is always invertible.  This method is also more computationally efficient. Sklearn ueses this method natively </b>

## Gradient Descent

<b> Gradient Descent outperforms sklearn and the Normal Equation when the dataset is extremely large (100,000+ features).  </b>

<b> General notes about gradient descent </b>
* Tweaks parameters iteratively in order to minimize a cost function
* Not all cost functions are convex.  As such, one hurdle for gradient descent is overcoming local minima and plateaus
* The learning rate is the primary parameter for correctly finding the absolute minima.  Too high and the optimizer will diverge and never find a minima. Too low and it will stop too soon at a local minima
* Because a convex function has exactly 1 minima they are ideal. If possible, transform the cost function such that its transformation is convex.
* Scaling all features has an enormous impact on how long it takes for the optimizer to find the mininma. 
* Gradient descent utilizes the properties of partial derivatives.  As such, a less complex derivative is arguably more important than a convex, continuous function.  Both is ideal.

## Gradient Vector of the Cost Function

#### $\nabla_{\boldsymbol\theta}MSE(\boldsymbol\theta) = \frac{2}{m}\mathbf{X}^{T}(\mathbf{X}\boldsymbol\theta - \mathbf{y})$

<br>

<b> Notice that this formula computes over the full training set X at each step.  The batch size here is 1 and as a result the training time on large datasets will be greatly impacted.  There are other ways to do this involving larger batch sizes which will be discussed later<b>

## Gradient Descent Step

#### $\boldsymbol\theta^{(next step)} = \boldsymbol\theta - \eta\nabla_{\boldsymbol\theta}MSE(\boldsymbol\theta)$

In [5]:
' Validate the above expressions'
eta = 0.1  #learning rate
n_iterations = 1000
m = 100

theta = np.random.randn(2, 1) # random initialization

for iteration in range(n_iterations):
    gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
    theta = theta - eta*gradients
    
print (theta, '\n', '\n', 'This is exactly what the Normal Equation yielded')


[[3.91900882]
 [3.27450507]] 
 
 This is exactly what the Normal Equation yielded


### Stochastic Gradient Descent

<b> Opposite to batch gradient descent which uses the entire training set at each epoch, stochastic gradient descent takes one random sample from the training set at each epoch and applies the gradient descent step.  This requires vastly less memory and as such is a better choice for very large training sets.  However, there are some nuances to the process.  Due to the randomness the descent curve will not be smooth and the optimal solution will never be settled upon.  This can help escape local minima but comes at the cost of a less perfect output.  One solution to converging on the optimal output using stochastic gradient descent is to implement a learning schedule in which the learning rate is gradually reduced such that the random jumps at the point of convergence are so small that the output is essentially the same as batch gradient descent. </b>

In [6]:
' An SGD example with a simple learning schedule'

# Epochs
n_epochs = 50

# Learning rate schedule
t0, t1 = 5, 50 

def learning_rate_schedule(t):
    return t0 / (t + t1)

# Random initialization
theta = np.random.randn(2, 1)

# SGD
for epoch in range(n_epochs):
    for i in range(m):  # m = 100 was declared in an earlier code block
        random_index = np.random.randint(m)
        xi = X_b[random_index: random_index + 1]
        yi = y[random_index: random_index + 1]
        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_rate_schedule(epoch * m + i)
        theta = theta - eta * gradients
        
theta

array([[3.91357171],
       [3.30173625]])

<b> When using SGD it is best practice to shuffle the training set </b>

In [7]:
' Implementing SGD with Sklearn'
from sklearn.linear_model import SGDRegressor
sgd_reg = SGDRegressor(max_iter=1000, tol=1e-3, penalty=None, eta0=0.1)
sgd_reg.fit(X, y.ravel()) # .ravel() is a numpy function which flattens an array into a single 1D array with all the input-array elements and with the same type as it

print(sgd_reg.intercept_, sgd_reg.coef_)

[3.85426458] [3.22932641]


### Mini-batch Gradient Descent

<b> Mini-batch gradient desecent is essentially the middle ground between batch and sgd.  It computes gradients on small, random sets of instances called mini-batches. One advantage of SGD is you can get performance boosts from hardward optimzation when using matrix operations and GPUs.  It will likely end up closer to the minimum than SGD as well, although it may be harder to escape from local minima.  For large datasets this is usually the best algorithm. <b>

### Polynomial Regression

<b> Suprisingly, you can use linear regression algorithms to model non-linear relationships.  For example, consider a simple parabola.  You can simply augment the square of the feature(s) as additional features into the dataset and viola, you can use linear regression to estimate non-linear relationships <b>

In [12]:
' An example of estimating a polynomial function with linear regression'
m = 100
X = 6 * np.random.rand(m, 1) - 3
y = 0.5 * X**2 + X + 2 + np.random.randn(m, 1)

from sklearn.preprocessing import PolynomialFeatures

poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly_features.fit_transform(X)

X[0], X_poly[0]

(array([-0.85321989]), array([-0.85321989,  0.72798418]))

In [13]:
lin_reg = LinearRegression()
lin_reg.fit(X_poly, y)
lin_reg.intercept_, lin_reg.coef_

# Here we see an intercept close to 2, a second-degree coeffecient close to 0.5, and a first-degree coeffecient close to 1 as expected

(array([2.13977325]), array([[1.0366274 , 0.50755395]]))

<b> Note that the PolynomialFeatures(degree=d) class will transform an array containing n-features into an array containing $\large\frac{(n + d)!}{d!n!}$ features.  Be cautious of the combinatorix nature of this augmentation <b>

### Learning Curves